# Using DirectView and LoadBalancedView together

We mentioned that you can use any number of views of any sort simultaneously.

The most common model for doing this is to use a DirectView to initialise the engine namespaces
(either defining functions and variables, or loading datasets) with a DirectView,
and then submitting work as tasks via the LoadBalancedView.

For this example, we are going to define a few functions for fetching data and getting a simple summary on some stocks,
then we are going to run those functions on a series of stocks in a load-balanced way.

The usual boilerplate:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import ipyparallel as parallel

rc = parallel.Client()
all_engines = rc[:]
lbv = rc.load_balanced_view()

Use matplotlib to fetch data from Yahoo

Note that we are defining these functions on the engines

In [ ]:
%%px

from io import BytesIO

from matplotlib import finance

def get_yahoo_data(name=None, start=None, end=None):
    """return historical stock data from yahoo as a recarray."""
    fp = finance.fetch_historical_yahoo(name, start, end)
    return finance.parse_yahoo_historical_ochl(fp, asobject=True)

def relative_close(records):
    """return the reative change in closing price over the interval"""
    
    close = records['aclose']
    return (close[-1] - close[0]) / close[0]



Setup the timestamps.  We want the past year

In [ ]:
%%px
import datetime
end = datetime.datetime.now()
start = end - datetime.timedelta(days=365)

Our ticker of stocks to check

In [ ]:
ticker = ['AAPL', 'GOOG', 'MSFT', 'ORCL', 'AMZN', 'INTC', 'ATVI', 'EA', 'NFLX']

And map a simple function (that depends on the code we have defined on the engines)
onto our ticker.

In [ ]:
def year_over_year(name):
    """fetch data, and return year-over-year change (relative to first close)
 
    in percent
    """
    stock = get_yahoo_data(name, start, end)
    return relative_close(stock) * 100

amr = lbv.map(year_over_year, ticker)
amr

In [ ]:
fig, ax = plt.subplots()
fig.set_figwidth(10)
ax.bar(range(len(amr)), amr)
ax.set_xticks(np.arange(len(amr))+0.4)
ax.set_xticklabels(ticker)
ax.axhline(0, c='k');

# Plotting stock data in parallel with Pandas

In [ ]:
%%px
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
try:
    from pandas_datareader import DataReader
except ImportError:
    from pandas.io.data import DataReader
from datetime import datetime

In [ ]:
def plot_stock_since(name, since_when, key="Adj Close"):
    data = DataReader(name, "yahoo", since_when)
    data[key].plot()
    plt.title(name)
    plt.show()
    

In [ ]:
start = datetime(2011,1,1)
plot_stock_since('GOOG', start)

In [ ]:
%%px
try:
    from pandas_datareader import DataReader
except ImportError:
    from pandas.io.data import DataReader

In [ ]:
all_engines.scatter('ticker', ticker)
%px print(ticker)

In [ ]:
all_engines['plot_stock_since'] = plot_stock_since

In [ ]:
%%px
from datetime import datetime
start = datetime(2011,1,1)
from pandas.io.data import DataReader

for stock in ticker:
    plot_stock_since(stock, start)

In [ ]:
amr = lbv.map(plot_stock_since, ticker, [start]*len(ticker))
amr

In [ ]:
%time amr.wait()
amr.display_outputs()